# Component Test: Train Model Distributed

## Author
- Sebastian Lehrig <sebastian.lehrig1@ibm.com>

## License
Apache-2.0 License

## Imports & Constants

In [30]:
import kfp
import kfp.dsl as dsl

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [31]:
BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"
KFP_CLIENT = kfp.Client()

with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace") as f:
    NAMESPACE = f.read()
NAMESPACE

'user-example-com'

## Specify training

References:
- https://horovod.readthedocs.io/en/stable/keras.html
- https://github.com/horovod/horovod/blob/master/examples/tensorflow2/tensorflow2_keras_mnist.py

In [53]:
def train_model():
    import tensorflow as tf
    import horovod.tensorflow.keras as hvd
    import os
    import time

    seconds = 10
    print(f"Sleeping {seconds} seconds...")
    time.sleep(seconds)
    print("Continuing...")

    # ptxas fix
    # TODO Remove cudatoolkit-dev install once part of base image
    import subprocess

    subprocess.run(["mamba", "install", "cudatoolkit-dev", "--yes", "--quiet"])

    # Horovod: initialize Horovod.
    os.environ["NCCL_DEBUG"] = "INFO"
    hvd.init()

    # Horovod: pin GPU to be used to process local rank (one GPU per process)
    gpus = tf.config.experimental.list_physical_devices("GPU")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], "GPU")

    (mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data(
        path="mnist-%d.npz" % hvd.rank()
    )

    dataset = tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(mnist_images[..., tf.newaxis] / 255.0, tf.float32),
            tf.cast(mnist_labels, tf.int64),
        )
    )
    dataset = dataset.repeat().shuffle(10000).batch(128)

    mnist_model = tf.keras.Sequential(
        [
            tf.keras.layers.Conv2D(32, [3, 3], activation="relu"),
            tf.keras.layers.Conv2D(64, [3, 3], activation="relu"),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

    # Horovod: adjust learning rate based on number of GPUs.
    scaled_lr = 0.001 * hvd.size()
    opt = tf.optimizers.Adam(scaled_lr)

    # Horovod: add Horovod DistributedOptimizer.
    opt = hvd.DistributedOptimizer(
        opt, backward_passes_per_step=1, average_aggregated_gradients=True
    )

    # Horovod: Specify `experimental_run_tf_function=False` to ensure TensorFlow
    # uses hvd.DistributedOptimizer() to compute gradients.
    mnist_model.compile(
        loss=tf.losses.SparseCategoricalCrossentropy(),
        optimizer=opt,
        metrics=["accuracy"],
        experimental_run_tf_function=False,
    )

    callbacks = [
        # Horovod: broadcast initial variable states from rank 0 to all other processes.
        # This is necessary to ensure consistent initialization of all workers when
        # training is started with random weights or restored from a checkpoint.
        hvd.callbacks.BroadcastGlobalVariablesCallback(0),
        # Horovod: average metrics among workers at the end of every epoch.
        #
        # Note: This callback must be in the list before the ReduceLROnPlateau,
        # TensorBoard or other metrics-based callbacks.
        hvd.callbacks.MetricAverageCallback(),
        # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
        # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
        # the first three epochs. See https://arxiv.org/abs/1706.02677 for details.
        hvd.callbacks.LearningRateWarmupCallback(
            initial_lr=scaled_lr, warmup_epochs=3, verbose=1
        ),
    ]

    # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
    if hvd.rank() == 0:
        callbacks.append(tf.keras.callbacks.ModelCheckpoint("./checkpoint-{epoch}.h5"))

    # Horovod: write logs on worker 0.
    verbose = 1 if hvd.rank() == 0 else 0

    # Train the model.
    # Horovod: adjust number of steps based on number of GPUs.
    mnist_model.fit(
        dataset,
        steps_per_epoch=500 // hvd.size(),
        callbacks=callbacks,
        epochs=24,
        verbose=verbose,
    )


train_specification = kfp.components.func_to_component_text(func=train_model)

In [47]:
train_comp = kfp.components.load_component_from_file("component.yaml")

## Create pipeline

TODO: Fix this issue when setting `worker = 2` (`ncclCommInitRank failed: unhandled cuda error`)

In [52]:
@dsl.pipeline(
    name="Component Test - Train Model Distributed",
    description="A simple component test",
)
def train_pipeline():
    worker = 1
    gpus = 1

    mpi_specification = {
        "distribution_type": "MPI",
        "number_of_workers": worker,
        "worker_cpus": "8",
        "worker_memory": "32Gi",
        "launcher_cpus": "2",
        "launcher_memory": "2Gi",
    }

    train_comp(
        train_dataset_dir="/tmp",
        validation_dataset_dir="/tmp",
        train_specification=train_specification,
        train_parameters={},
        gpus=gpus,
        distribution_specification=mpi_specification,
    ).set_display_name(f"Distributed MPIJob - {worker} worker x {gpus} GPUs")

## Run the pipeline within an experiment

In [51]:
KFP_CLIENT.create_run_from_pipeline_func(
    train_pipeline, arguments={}, namespace=NAMESPACE
)

RunPipelineResult(run_id=acadd34e-04ce-467f-b032-90147d1d5e63)